In [ ]:
!pip install pyngrok
!ngrok authtoken 2nkl6K2j1sBKduiqbI8wK12OZ7b_5kMfodVD2ugziLMNL1q66

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# Step 1: Install necessary libraries
!pip install flask
!pip install jinja2==3.0.3
!pip install itsdangerous==2.0.1
!pip install pyngrok
!pip install ibm_db
!pip install pandas

# Step 2: Import necessary libraries
from flask import Flask, render_template_string
from pyngrok import ngrok
import ibm_db
import pandas as pd
import os

# IBM Db2 credentials (use your credentials here)
dsn_hostname = os.getenv('DB_HOST', '815fa4db-dc03-4c70-869a-a9cc13f33084.bs2io90l08kqb1od8lcg.databases.appdomain.cloud')
dsn_uid = os.getenv('DB_UID', 'drl64387')
dsn_pwd = os.getenv('DB_PWD', 'nCd2UC0r6DXLjRrz')
dsn_port = os.getenv('DB_PORT', '30367')
dsn_database = "bludb"
dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_protocol = "TCPIP"
dsn_security = "SSL"

# Create the dsn connection string
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};"
    "SECURITY={7};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd, dsn_security)

# Step 3: Define the Flask app
app = Flask(__name__)

# Function to fetch data from the database and return it as a DataFrame
def fetch_data_from_db():
    try:
        # Connect to the IBM Db2 database
        conn = ibm_db.connect(dsn, "", "")
        print("Connected to database:", dsn_database, "as user:", dsn_uid, "on host:", dsn_hostname)

        # SQL query to fetch all rows from STUDENT_LIST
        command = "SELECT * FROM STUDENT_LIST FETCH FIRST 10 ROWS ONLY"  # Modify query as needed
        stmt = ibm_db.exec_immediate(conn, command)

        # Fetch results and store in a list
        rows = []
        result = ibm_db.fetch_both(stmt)
        while result:
            rows.append(result)
            result = ibm_db.fetch_both(stmt)

        # Convert results to a pandas DataFrame
        df = pd.DataFrame(rows)
        if df.empty:
            print("Warning: No data fetched from the database.")
        return df

    except Exception as e:
        print("Error connecting to the database:", str(e))
        return pd.DataFrame()  # Return an empty DataFrame in case of error

    finally:
        # Close the connection
        if 'conn' in locals() and conn:
            ibm_db.close(conn)

# Step 4: Define the route to render the HTML template
@app.route('/')
def home():
    try:
        # Fetch data from the database
        df = fetch_data_from_db()

        # Convert the DataFrame to HTML table
        table_html = df.to_html(classes='table table-bordered', index=False)

        # Define the HTML template with the data table and Kommunicate chatbot
        html_template = """
        <!DOCTYPE html>
        <html>
        <head>
            <title>IBM Db2 Data and Kommunicate Chatbot</title>
            <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css">
        </head>
        <body>
            <div class="container mt-4">
                <h1 class="text-center">Data from IBM Db2 Database</h1>
                {table_html}
            </div>

            <!-- Kommunicate Script -->
            <script type="text/javascript">
                (function(d, m){{
                    var kommunicateSettings =
                        {{"appId":"104c53a09b4a2a5ee790342ff70f9136d","popupWidget":true,"automaticChatOpenOnNavigation":true}};
                    var s = document.createElement("script"); s.type = "text/javascript"; s.async = true;
                    s.src = "https://widget.kommunicate.io/v2/kommunicate.app";
                    var h = document.getElementsByTagName("head")[0]; h.appendChild(s);
                    window.kommunicate = m; m._globals = kommunicateSettings;
                }})(document, window.kommunicate || {});
            </script>
        </body>
        </html>
        """.replace("{table_html}", table_html)  # Replacing {table_html} with actual HTML table

        return render_template_string(html_template)

    except Exception as e:
        # Log error message
        error_message = f"Error rendering page: {str(e)}"
        print(error_message)
        return f"<h1>Internal Server Error</h1><p>{error_message}</p>", 500

# Step 5: Run the Flask app with ngrok
def run_app():
    # Start ngrok tunnel
    public_url = ngrok.connect(5000)
    print(f" * Ngrok Tunnel URL: {public_url}")

    # Run the Flask app
    app.run(port=5000, debug=True)

# Start the app
run_app()


 * Ngrok Tunnel URL: NgrokTunnel: "https://6237-34-66-44-192.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [22/Oct/2024 10:11:06] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Oct/2024 10:11:07] "GET /favicon.ico HTTP/1.1" 404 -
